# Lab-10-3 visdom

### 10-3-1 Visdom Example

In [41]:
import torch
import torch.nn as nn

import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

### Import visdom

In [2]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


### Text

In [3]:
vis.text('Hello, world!', env = 'main')

'window_3a59824da14b12'

### Image

In [6]:
a = torch.randn(3, 200, 200)
vis.image(a)

'window_3a59829ae4e90c'

### Images

In [7]:
vis.images(torch.Tensor(3,3,28,28))

'window_3a5982c86bf7bc'

### Example (using MNIST and CIFAR10)

In [9]:
MNIST = dsets.MNIST(root = './MNIST_data', train = True, transform = torchvision.transforms.ToTensor(), download = True)
cifar10 = dsets.CIFAR10(root = './cifar10', train = True, transform = torchvision.transforms.ToTensor(), download = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST_data\MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST_data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST_data\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST_data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST_data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST_data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST_data\MNIST\raw



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10\cifar-10-python.tar.gz to ./cifar10


### CIFAR10

In [10]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([3, 32, 32])


'window_3a59845b55d9c8'

### MNIST

In [11]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([1, 28, 28])


'window_3a598490da386a'

### Check dataset

In [14]:
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                          batch_size = 32,
                                          shuffle = False)

In [15]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


### Line Plot

In [20]:
Y_data = torch.randn(5)
plt = vis.line(Y=Y_data)

In [21]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

### Line update

In [25]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update ='append')

'window_3a5985848d38e0'

### Multiple Line on single windows

In [28]:
num = torch.Tensor(list(range(0, 10)))
num = num.view(-1,1)
num = torch.cat((num,num), dim=1)

plt = vis.line(Y=torch.randn(10,2), X = num)

### Line info

In [31]:
plt = vis.line(Y = Y_data, X = X_data, opts = dict(title='Test', showlegend=True))

In [36]:
plt = vis.line(Y = Y_data, X = X_data, opts = dict(title='Test', legend = ['1번'], showlegend=True))

In [37]:
plt = vis.line(Y = torch.randn(10,2), X = num, opts = dict(title='Test', legend=['1번',  '2번'],showlegend=True))

### make function for update line

In [38]:
def loss_tracker(loss_plot, loss_value, num):
    # num, loss_value are Tensor
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
            )

In [39]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1)+i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [40]:
vis.close(env = 'main')

''

# 10-3 MNIST-CNN with Visdom

In [42]:
import torch
import torch.nn as nn

import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

### Import visdom

In [59]:
import visdom

vis = visdom.Visdom()
vis.close(env='main')

Setting up a new session...


''

### Define loss_tracker

In [60]:
def loss_tracker(loss_plot, loss_value, num):
    # num, loss_value are Tensor
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
            )

### CNN

In [45]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [46]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [47]:
# MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [48]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [49]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__() 
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride=1, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
            
            
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        
        self.fc1 = nn.Linear(3*3*128, 625) 
        self.relu = nn.ReLU()        
        self.fc2 = nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [50]:
model = CNN().to(device)

In [51]:
model
value = torch.Tensor(1,1,28,28).to(device)
print( (model(value)) )

tensor([[ 5.1100e+16,  3.8152e+17, -6.1219e+17,  2.9240e+17,  1.3782e+17,
          4.7368e+17, -1.0012e+18, -3.8013e+16,  8.3217e+16,  9.5133e+17]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [52]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

### Make plot

In [53]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts = dict(title='loss tracker', legend=['loss'], showlegend=True))

### Train with loss_tracker

In [61]:
# training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost +=cost / total_batch
        
    print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished')

[Epoch: 1] cost = 0.005113804712891579
[Epoch: 2] cost = 0.0055962735787034035
[Epoch: 3] cost = 0.004859673324972391
[Epoch: 4] cost = 0.0032735804561525583
[Epoch: 5] cost = 0.004115603398531675
[Epoch: 6] cost = 0.006070964504033327
[Epoch: 7] cost = 0.003521468723192811
[Epoch: 8] cost = 0.004566838964819908
[Epoch: 9] cost = 0.002439166186377406
[Epoch: 10] cost = 0.004232680890709162
[Epoch: 11] cost = 0.0020350124686956406
[Epoch: 12] cost = 0.0026155486702919006
[Epoch: 13] cost = 0.004212360829114914
[Epoch: 14] cost = 0.0015908972127363086
[Epoch: 15] cost = 0.0034409677609801292
Learning Finished


In [62]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9869999885559082
